In [ ]:
sudo docker run -dit --name nvidia-10.1-container --gpus all nvidia/cuda:10.1-devel-ubuntu18.04 bash

In [ ]:
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [ ]:
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
double x=0.0;

if(threadIdx.x<=n-1){
x=a+(threadIdx.x+1/2.0)*h_hat;
data[threadIdx.x]=std::exp(-std::pow(x,2));
}
    *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
  //*sum = thrust::reduce(thrust::seq, data , data + n, (double)0, thrust::plus<double>());
}


int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    return 0;
}

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01

Error relativo de la solución: 4.104931878976858e-08